<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# EPAT Session 1

**Executive Program in Algorithmic Trading**

**_Vectorized Backtesting_**

Prof. Dr. Yves J. Hilpisch | The Python Quants GmbH | http://tpq.io

<a href="https://home.tpq.io/certificates/pyalgo" target="_blank"><img src="https://hilpisch.com/pyalgo_cover_shadow.png" width="300px" align="left"></a>

## Basic Imports

In [ ]:
import cufflinks
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment', None)
cufflinks.set_config_file(offline=True)
%config InlineBackend.figure_format = 'svg'

## Reading Financial Data

In [ ]:
url = 'http://hilpisch.com/pyalgo_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
raw.head()

In [ ]:
raw['RANDOM'] = 100
raw['RANDOM'].iloc[1:] += np.random.standard_normal(len(raw) - 1).cumsum()

In [ ]:
raw['AAPL.O'].plot();

In [ ]:
raw['RANDOM'].plot();

## Efficient Markets

In [ ]:
symbol = 'RANDOM'
symbol = 'AAPL.O'
symbol = '.SPX'
symbol = 'EUR='

In [ ]:
data = pd.DataFrame(raw[symbol])

In [ ]:
data.head()

In [ ]:
lags = 5

In [ ]:
cols = list()
for lag in range(1, lags + 1):
    col = f'lag_{lag}'
    data[col] = data[symbol].shift(lag)
    cols.append(col)

In [ ]:
data.head(7)

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.head()

In [ ]:
reg = np.linalg.lstsq(data[cols], data[symbol], rcond=-1)[0]

In [ ]:
reg

In [ ]:
data[cols].corr()

## Simple Trading Strategy

In [ ]:
symbol = 'RANDOM'
symbol = 'AAPL.O'
symbol = '.SPX'
symbol = 'EUR='

In [ ]:
data = pd.DataFrame(raw[symbol])

In [ ]:
data['SMA1'] = data[symbol].rolling(42).mean()

In [ ]:
data['SMA2'] = data[symbol].rolling(252).mean()

In [ ]:
data.plot();

In [ ]:
data.dropna(inplace=True)

In [ ]:
# data['p'] = np.where(data['SMA1'] > data['SMA2'], 'long', 'short')

In [ ]:
data['p'] = np.where(data['SMA1'] > data['SMA2'], 1, -1)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.plot(secondary_y='p');

## Vectorized Backtesting

In [ ]:
data['r'] = np.log(data[symbol] / data[symbol].shift(1))  # log returns

In [ ]:
data.head()

In [ ]:
data['s'] = data['p'].shift(1) * data['r']

In [ ]:
data[['r', 's']].sum()  # sum of log returns

In [ ]:
data[['r', 's']].sum().apply(np.exp)  # gross performance

In [ ]:
data[['r', 's']].cumsum().apply(np.exp).plot();  # gross performance over time

## Excursion: Discounting

In [ ]:
S0 = 100
S1 = 110

In [ ]:
r = S1 / S0 - 1

In [ ]:
r

In [ ]:
r = (S1 - S0) / S0  # simple return

In [ ]:
r

In [ ]:
import math

In [ ]:
lr = math.log(S1 / S0)  # log return

In [ ]:
lr

In [ ]:
(1 + r) * S0  # simple return

In [ ]:
math.exp(lr)

In [ ]:
math.exp(lr) * S0  # log return

## Optimizing the Parameters (OOP Way)

### `FinancialData` Class

In [ ]:
class FinancialData:
    url = 'http://hilpisch.com/pyalgo_eikon_eod_data.csv'
    def __init__(self, symbol):
        self.symbol = symbol
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col=0,
                               parse_dates=True).dropna()
    def prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data['r'] = np.log(self.data / self.data.shift(1))
    def plot_data(self, cols=None):
        if cols is None:
            cols = [self.symbol,]
        self.data[cols].plot()

In [ ]:
fd = FinancialData('EUR=')  # instantiation

In [ ]:
fd.url  # class attribute

In [ ]:
fd.symbol  # instance attribute

In [ ]:
# fd.retrieve_data()  # method call

In [ ]:
fd.data.head()

In [ ]:
fd.plot_data()

### Vectorized Backtesting Class

In [ ]:
class SMABacktester(FinancialData):
    def prepare_statistics(self):
        self.data['SMA1'] = self.data[self.symbol].rolling(self.SMA1).mean()
        self.data['SMA2'] = self.data[self.symbol].rolling(self.SMA2).mean()
    def backtest_strategy(self, SMA1, SMA2, start=None):
        self.SMA1 = SMA1
        self.SMA2 = SMA2
        self.prepare_statistics()
        if start is None:
            start = self.SMA2
        self.data['p'] = np.where(self.data['SMA1'] > self.data['SMA2'], 1, -1)
        self.data['s'] = self.data['p'].shift(1) * self.data['r']
        perf = self.data[['r', 's']].iloc[start:].sum().apply(np.exp)
        return perf
    def plot_results(self):
        self.data[['r', 's']].iloc[self.SMA2:].cumsum().apply(np.exp).plot()

In [ ]:
sma = SMABacktester('EUR=')
# sma = SMABacktester('GLD')
# sma = SMABacktester('GS.N')

In [ ]:
sma.data.info()

In [ ]:
sma.backtest_strategy(42, 252)

In [ ]:
sma.data.info()

In [ ]:
sma.plot_data([sma.symbol, 'SMA1', 'SMA2'])

In [ ]:
sma.plot_results()

### Optimization

In [ ]:
from itertools import product

In [ ]:
sma1 = [30, 40, 50]

In [ ]:
sma2 = [200, 240]

In [ ]:
product(sma1, sma2)

In [ ]:
list(product(sma1, sma2))

In [ ]:
for SMA1, SMA2 in product(sma1, sma2):
    perf = sma.backtest_strategy(SMA1, SMA2)
    print(SMA1, SMA2, '\n', perf)

In [ ]:
sma.plot_results()

## Optimization within Class

Optimization in the sense of "brute force" optimization (= checking a large number of parameter combinations and picking the best one).

In [ ]:
class SMABacktester(SMABacktester):
    def optimize_parameters(self, sma1, sma2):
        start = max(sma2)
        self.res = pd.DataFrame()
        for SMA1, SMA2 in product(sma1, sma2):
            perf = sma.backtest_strategy(SMA1, SMA2, start)
            self.df = pd.DataFrame({'SMA1': SMA1, 'SMA2': SMA2,
                                   'BENCH': perf['r'], 'STRAT': perf['s'],
                                   'OUTPERF': perf['s'] - perf['r']},
                                  index=[0,])
            self.res = self.res.append(self.df, ignore_index=True)

In [ ]:
sma = SMABacktester('EUR=')

In [ ]:
sma.optimize_parameters(sma1, sma2)

In [ ]:
sma.res

In [ ]:
sma.res.sort_values('OUTPERF', ascending=False)

In [ ]:
sma.res.groupby('SMA1').mean()[['STRAT', 'OUTPERF']]

In [ ]:
sma.res.groupby('SMA2').mean()[['STRAT', 'OUTPERF']]

## Another Example

In [ ]:
sma1 = range(30, 61, 1)

In [ ]:
sma2 = range(150, 301, 2)

In [ ]:
%time sma.optimize_parameters(sma1, sma2)

In [ ]:
len(sma.res)

In [ ]:
sma.res.head()

In [ ]:
sma.res.sort_values('OUTPERF', ascending=False).head()

In [ ]:
sma.res.groupby('SMA1').mean()[['STRAT', 'OUTPERF']].sort_values(
    'OUTPERF', ascending=False).head()

In [ ]:
sma.res.groupby('SMA2').mean()[['STRAT', 'OUTPERF']].sort_values(
    'OUTPERF', ascending=False).head()

<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>